- https://github.com/tensorflow/models/tree/master/research/im2txt
- https://yashk2810.github.io/
- https://github.com/yashk2810/Image-Captioning/blob/master/Image%20Captioning%20InceptionV3.ipynb
- https://arxiv.org/pdf/1609.06647.pdf

In [1]:
from PIL import Image
from glob import glob
import os
import re
import pdb
import pickle

In [2]:
dpath = '/data/image_captioning/flikr8/'
img_path = dpath+'Flicker8k_Dataset/'


## Caption Processing

First, maybe just take the first caption for each image.
- construct a regex that matches the first captions

### Create file of only first captions

In [3]:
match = '1003163366_44323f5815.jpg#0 '
match_2 = '1003163366_44323f5815.jpg#2 '

In [4]:
first_cap_re = re.compile('^.+\.jpg#\d')

In [5]:
first_cap_re.match(match), first_cap_re.match(match_2)

(<_sre.SRE_Match object; span=(0, 27), match='1003163366_44323f5815.jpg#0'>,
 <_sre.SRE_Match object; span=(0, 27), match='1003163366_44323f5815.jpg#2'>)

In [7]:
all_caps_path = dpath+'Flickr8k.token.txt'
all_caps = []
for line in open(all_caps_path):
    if first_cap_re.match(line):
        line_arr = line.split('\t')
        img_nm = line_arr[0][:-2]
        cap = line_arr[1].strip()
        all_caps.append((img_nm, cap))

In [8]:
len(all_caps)

40455

In [12]:
all_caps[2]

('1000268201_693b08cb0e.jpg',
 'A little girl climbing into a wooden playhouse .')

In [13]:
pickle.dump(all_caps, open(dpath+'all_caps_2.pkl', 'wb'))

In [40]:
all_caps = pickle.load(open(dpath+'all_caps.pkl', 'rb'));

### Separate train, dev and test captions

In [15]:
def getImgCapForSet(set_names_path, all_imgs_caps):    
    nms = [line.strip() for line in open(set_names_path)]
    return [name_cap for name_cap in all_imgs_caps if name_cap[0] in nms] 


In [16]:
tr_nms_caps = getImgCapForSet(dpath+'Flickr_8k.trainImages.txt', all_caps)

In [21]:
len(tr_nms_caps), tr_nms_caps[99]

(30000,
 ('1024138940_f1fefbdce1.jpg',
  'Two large tan dogs play along a sandy beach .'))

In [23]:
test_nms_caps = getImgCapForSet(dpath+'Flickr_8k.testImages.txt', all_caps)
len(test_nms_caps), test_nms_caps[100]

(5000,
 ('1258913059_07c613f7ff.jpg',
  'A couple of people sit outdoors at a table with an umbrella and talk .'))

In [24]:
dev_nms_caps = getImgCapForSet(dpath+'Flickr_8k.devImages.txt', all_caps)
len(dev_nms_caps), dev_nms_caps[100]

(5000,
 ('1215334959_b1970965f7.jpg', 'A family sits on a bench near a beach .'))

In [25]:
pickle.dump(tr_nms_caps, open(dpath+'train_all_caps_2.pkl', 'wb'))
pickle.dump(test_nms_caps, open(dpath+'test_all_caps_2.pkl', 'wb'))
pickle.dump(dev_nms_caps, open(dpath+'dev_all_caps_2.pkl', 'wb'))

### Move Images

In [5]:
tr_img_names = [nm.strip() for nm in open(dpath+ 'Flickr_8k.trainImages.txt')]

tr_img_path = img_path+'train'
for nm in tr_img_names:
    os.rename(os.path.join(img_path, nm), os.path.join(tr_img_path, nm))

In [20]:
def moveImgs(nm_path, out_path, img_path=img_path):
    img_names = [nm.strip() for nm in open(nm_path)]
    for nm in img_names:
        os.rename(os.path.join(img_path, nm), os.path.join(out_path, nm))

In [21]:
moveImgs(dpath+ 'Flickr_8k.devImages.txt', img_path+'dev')

In [24]:
moveImgs(dpath+ 'Flickr_8k.testImages.txt', img_path+'test')

### Create Sample

In [6]:
from shutil import copy

In [5]:
samp_path =img_path+'samp/'

In [7]:
tr_imgs = glob(img_path+'train/wrap/*.jpg')

In [8]:
first32 = tr_imgs[:32]

In [13]:
next32 = tr_imgs[32:65]

In [10]:
len(first32)

32

In [20]:
def copyToPath(fs, p):
    for f in fs:
        copy(f, os.path.join(p, os.path.basename(f)))

In [21]:
copyToPath(first32, samp_path+'train/wrap/')
copyToPath(next32, samp_path+'val/wrap/')
